In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 22586468
paper_name = 'peyroche_plateau_2012' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/journal.pone.0036343.s004.xlsx', sheet_name='data')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4520 x 12


In [7]:
original_data.head()

,ORF,NAME,Synonym,d,"s (1 µM, 16 h)","s (2 µM, 16 h)","s (1 µM, 27 h)","s (2 µM, 27 h)",s,log2(s),ORF type,SGD description
0,YJL101C,GSH1,NaN,1.028368,0.157521,0.195512,0.399111,0.465400,0.297312,-1.749951,Verified,Gamma glutamylcysteine synthetase catalyzes th...
1,YDR004W,RAD57,NaN,0.897125,0.243763,0.373507,0.613142,0.092539,0.308635,-1.696026,Verified,Protein that stimulates strand exchange by sta...
2,YPL091W,GLR1,LPG17,1.050194,0.107089,0.178998,0.461548,0.444078,0.311538,-1.682520,Verified,Cytosolic and mitochondrial glutathione oxidor...
3,YJR139C,HOM6,NaN,0.916193,0.260004,0.238142,0.370637,0.599299,0.315320,-1.665109,Verified,Homoserine dehydrogenase (L-homoserine:NADP ox...
4,YNL107W,YAF9,NaN,0.970198,0.300925,0.350154,0.478594,0.259199,0.325540,-1.619094,Verified,Subunit of both the NuA4 histone H4 acetyltran...


In [8]:
original_data['orf'] = original_data['ORF'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, NAME, Synonym, d, s (1 µM, 16 h), s (2 µM, 16 h), s (1 µM, 27 h), s (2 µM, 27 h), s, log2(s), ORF type, SGD description, orf]
Index: []


In [12]:
original_data.set_index('orf', inplace=True)

In [13]:
original_data = original_data[['s (1 µM, 16 h)','s (2 µM, 16 h)','s (1 µM, 27 h)','s (2 µM, 27 h)']].copy()

In [14]:
original_data = original_data.groupby(original_data.index).mean()

In [15]:
original_data.shape

(4520, 4)

# Prepare the final dataset

In [16]:
data = original_data.copy()

In [17]:
dataset_ids = [57, 58, 525, 526]
datasets = datasets.reindex(index=dataset_ids)

In [18]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [19]:
data.head()

dataset_id,57,58,525,526
data_type,value,value,value,value
orf,,,,
YAL002W,0.796998,0.789628,0.854570,0.838197
YAL004W,1.053887,0.973368,1.010200,0.975580
YAL005C,0.913742,0.945826,0.932066,0.956808
YAL007C,0.989577,0.982781,1.002409,1.012799
YAL008W,0.956052,1.016716,1.002986,1.034916


## Subset to the genes currently in SGD

In [20]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 15


In [21]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,57,58,525,526
,data_type,value,value,value,value
gene_id,orf,,,,
2,YAL002W,0.796998,0.789628,0.854570,0.838197
1863,YAL004W,1.053887,0.973368,1.010200,0.975580
4,YAL005C,0.913742,0.945826,0.932066,0.956808
5,YAL007C,0.989577,0.982781,1.002409,1.012799
6,YAL008W,0.956052,1.016716,1.002986,1.034916


# Normalize

In [22]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [23]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [24]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,57,58,525,526,57,58,525,526
,data_type,value,value,value,value,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,
2,YAL002W,0.796998,0.789628,0.854570,0.838197,-1.390791,-1.451114,-1.317392,-1.576471
1863,YAL004W,1.053887,0.973368,1.010200,0.975580,0.306551,-0.174642,0.143331,-0.310917
4,YAL005C,0.913742,0.945826,0.932066,0.956808,-0.619428,-0.365983,-0.590021,-0.483844
5,YAL007C,0.989577,0.982781,1.002409,1.012799,-0.118366,-0.109251,0.070213,0.031946
6,YAL008W,0.956052,1.016716,1.002986,1.034916,-0.339873,0.126504,0.075620,0.235676


# Print out

In [25]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [26]:
from IO.save_data_to_db3 import *

In [27]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/4 [00:00<?, ?it/s]

Deleting all datasets for PMID 22586468...
Inserting the new data...


100%|██████████| 4/4 [00:29<00:00,  7.27s/it]

Updating the data_modified_on field...
